# Animals-10 Dataset
Download the raw data from: https://www.kaggle.com/datasets/alessiocorrado99/animals10?resource=download \
Then use function below to convert .pt for faster process

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import vit_b_16
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import random

In [3]:
# Define dataset paths
data_dir = "./animals10"
torch_dataset_path = "./animals10DatasetNoNormalize.pt"

# Define class names
class_names = ["cane", "cavallo", "elefante", "farfalla", "gallina", "gatto", "mucca", "pecora", "ragno", "scoiattolo"]
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Process and store images in Torch tensor format
if not os.path.exists(torch_dataset_path):
    print("Processing dataset and saving to Torch tensor format...")
    images, labels = [], []
    
    for class_name in class_names:
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('png', 'jpg', 'jpeg')):
                    img_path = os.path.join(class_path, img_name)
                    image = Image.open(img_path).convert("RGB")
                    image = transform(image)
                    images.append(image)
                    labels.append(class_to_idx[class_name])
    
    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)
    
    # Save to Torch tensor format
    torch.save({"images": images, "labels": labels}, torch_dataset_path)
    print("Dataset saved in Torch tensor format.")


Processing dataset and saving to Torch tensor format...
Dataset saved in Torch tensor format.


# Animals-90 Dataset
Download the raw data from: iamsouravbanerjee/animal-image-dataset-90-different-animals
Then use function below to convert .pt for faster process

In [2]:

import kagglehub

# Download latest version
test_path = kagglehub.dataset_download("naykrit/animal-classification")
train_path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")
train_dir = train_path + '/animals/animals'
test_dir = test_path + '/test_set'
print("Path to test files:", test_dir)
print("Path to train files:", train_dir)

c:\Users\Hizan\softwares\anaconda3\envs\ece1508\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to test files: C:\Users\Hizan\.cache\kagglehub\datasets\naykrit\animal-classification\versions\1/test_set
Path to train files: C:\Users\Hizan\.cache\kagglehub\datasets\iamsouravbanerjee\animal-image-dataset-90-different-animals\versions\5/animals/animals


In [3]:
def load_images_from_directory(directory, sel, save_path, image_size=(150, 150)):
    images = []
    labels = []
    transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                          std=[0.229, 0.224, 0.225])   # ImageNet stds
    ])

    for class_name in sel:
        print(f"Processing class: {class_name}")
        class_path = os.path.join(directory, class_name)
        print(f"Class path: {class_path}")
        # Check if the class directory exists
        
        if os.path.isdir(class_path):
            # Get all image files in the directory
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = Image.open(img_path).convert("RGB")
                    img_tensor = transform(img)
                    images.append(img_tensor)
                    labels.append(sel.index(class_name))

   
    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)

    # Save to Torch tensor format
    torch.save({"images": images, "labels": labels}, save_path)
    print("Dataset saved in Torch tensor format.")

In [4]:
def load_images_from_directory_with_augmentation(directory, sel, train_save_path, val_save_path, image_size=(224, 224)):
    image_label_pairs = []

    # Define transforms
    train_transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

    val_transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

    # Load image paths and labels
    for class_name in sel:
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    label = sel.index(class_name)
                    image_label_pairs.append((img_path, label))

    # Shuffle and split
    random.shuffle(image_label_pairs)
    total = len(image_label_pairs)
    val_size = int(total * 0.2)

    val_data = image_label_pairs[:val_size]
    train_data = image_label_pairs[val_size:]

    # Apply transforms and convert to tensors
    def process_data(data_list, transform):
        images = []
        labels = []
        for img_path, label in data_list:
            img = Image.open(img_path).convert("RGB")
            img_tensor = transform(img)
            images.append(img_tensor)
            labels.append(label)
        return torch.stack(images), torch.tensor(labels, dtype=torch.long)

    train_images, train_labels = process_data(train_data, train_transform)
    val_images, val_labels = process_data(val_data, val_transform)

    # Save
    torch.save({"images": train_images, "labels": train_labels}, train_save_path)
    torch.save({"images": val_images, "labels": val_labels}, val_save_path)

    print(f"Saved {len(train_images)} training samples to {train_save_path}")
    print(f"Saved {len(val_images)} validation samples to {val_save_path}")

In [ ]:
# Define dataset path to save
torch_train_path = "./animals90TrainDatasetArgumentation.pt"
torch_test_path = "./animals90TestDatasetArgumentation.pt"
torch_val_path = "./animals90ValDatasetArgumentation.pt"

class_names = [
    "antelope", "badger", "bat", "bear", "bee", "beetle", "bison", "boar", "butterfly",
    "cat", "caterpillar", "chimpanzee", "cockroach", "cow", "coyote", "crab", "crow",
    "deer", "dog", "dolphin", "donkey", "dragonfly", "duck", "eagle", "elephant",
    "flamingo", "fly", "fox", "goat", "goldfish", "goose", "gorilla", "grasshopper",
    "hamster", "hare", "hedgehog", "hippopotamus", "hornbill", "horse", "hummingbird",
    "hyena", "jellyfish", "kangaroo", "koala", "ladybugs", "leopard", "lion", "lizard",
    "lobster", "mosquito", "moth", "mouse", "octopus", "okapi", "orangutan", "otter",
    "owl", "ox", "oyster", "panda", "parrot", "pelecaniformes", "penguin", "pig",
    "pigeon", "porcupine", "possum", "raccoon", "rat", "reindeer", "rhinoceros",
    "sandpiper", "seahorse", "seal", "shark", "sheep", "snake", "sparrow", "squid",
    "squirrel", "starfish", "swan", "tiger", "turkey", "turtle", "whale", "wolf",
    "wombat", "woodpecker", "zebra"
]

load_images_from_directory_with_augmentation(train_dir, class_names, torch_train_path, torch_val_path)
load_images_from_directory(test_dir, class_names, torch_test_path)